# Fase 4: Busca de Parâmetros

Nesta fase iremos aprofundar a busca de hiperparamewtros, variabndo os parametros ca cada modelos, e compaerandos seus resultados

O primeiro passo que iremos fazer é importar as bibliotecas principais, e estabelecer o ambiente de execução.


In [1]:
# Bibliotecas Externas
import os
import warnings
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scipy.stats import randint, uniform

# Ignorando Warnings
warnings.filterwarnings("ignore")

# Bibliotecas Locais
os.chdir("../..")
from lib import plots, runner

# Definido Variáveis globais
N_SPLITS = 5
RANDOM_STATE = 51

## 0. Importando os Dados

Nosso primero é coletar os dados que preparamos na etapa anterior, para verificar como esses dados se comportam

In [2]:
# Conjunto de treino
X_train = pd.read_csv('./data/processed/X_train.csv')
Y_train = pd.read_csv('./data/processed/Y_train.csv')

Y_train['class'] = Y_train['class'].apply(lambda val: 1 if val == 'UP' else 0)

# Conjunto de validação
X_val = pd.read_csv('./data/processed/X_val.csv')
Y_val = pd.read_csv('./data/processed/Y_val.csv')


Y_val['class'] = Y_val['class'].apply(lambda val: 1 if val == 'UP' else 0)

# Conjunto de teste
X_test = pd.read_csv('./data/processed/X_test.csv')
Y_test = pd.read_csv('./data/processed/Y_test.csv')

Y_test['class'] = Y_test['class'].apply(lambda val: 1 if val == 'UP' else 0)

## 2. Algoritmos de Machine Learning para Classificação

### Comitê de Redes Neurais (com redes neurais definidas previamente)

In [19]:
# Definição das redes neurais com diferentes hiperparâmetros
mlp1 = MLPClassifier(hidden_layer_sizes=(50,), activation='relu', solver='adam', random_state=RANDOM_STATE)
mlp2 = MLPClassifier(hidden_layer_sizes=(100,), activation='tanh', solver='sgd', random_state=RANDOM_STATE)
mlp3 = MLPClassifier(hidden_layer_sizes=(50, 50), activation='relu', solver='lbfgs', random_state=RANDOM_STATE)
mlp4 = MLPClassifier(hidden_layer_sizes=(200,), activation='logistic', solver='adam', random_state=RANDOM_STATE)
mlp5 = MLPClassifier(hidden_layer_sizes=(100, 50), activation='tanh', solver='sgd', random_state=RANDOM_STATE)

In [20]:
# Criando o comitê com votação majoritária
voting_clf = VotingClassifier(estimators=[
    ('mlp1', mlp1),
    ('mlp2', mlp2),
    ('mlp3', mlp3),
    ('mlp4', mlp4),
    ('mlp5', mlp5)
], voting='hard')

In [21]:
# Treinando o comitê
voting_clf.fit(X_train, Y_train.values.ravel())

VotingClassifier(estimators=[('mlp1',
                              MLPClassifier(hidden_layer_sizes=(50,),
                                            random_state=51)),
                             ('mlp2',
                              MLPClassifier(activation='tanh', random_state=51,
                                            solver='sgd')),
                             ('mlp3',
                              MLPClassifier(hidden_layer_sizes=(50, 50),
                                            random_state=51, solver='lbfgs')),
                             ('mlp4',
                              MLPClassifier(activation='logistic',
                                            hidden_layer_sizes=(200,),
                                            random_state=51)),
                             ('mlp5',
                              MLPClassifier(activation='tanh',
                                            hidden_layer_sizes=(100, 50),
                                            random_state=51, solver='sgd'))])

In [22]:
# Fazendo previsões
Y_pred = voting_clf.predict(X_val)

In [23]:
# Avaliação do modelo
accuracy = accuracy_score(Y_val, Y_pred)
print(f'Acurácia do Comitê de Redes Neurais: {accuracy:.4f}')

Acurácia do Comitê de Redes Neurais: 0.7452


### Usando RandomizedSearchCV

In [6]:
# Definição do espaço de busca para os hiperparâmetros
param_dist = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (200,), (100, 50), (150,), (75, 75), (300,), (50, 100, 50)],
    'activation': ['relu', 'tanh', 'logistic', 'identity'],
    'solver': ['adam', 'sgd', 'lbfgs'],
    'alpha': uniform(0.0001, 0.1),  
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'max_iter': randint(200, 1000)
}


In [7]:
# Configuração da validação cruzada estratificada k-fold = 10
cv_strategy = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

In [8]:
performance_history = []
best_mlps = []

In [ ]:
# Aplicando RandomizedSearchCV para encontrar os melhores parâmetros para cada MLP
for i in range(5):
    mlp = MLPClassifier(random_state=RANDOM_STATE+i)
    rand_search = RandomizedSearchCV(mlp, param_dist, n_iter=20, cv=cv_strategy, n_jobs=-1, random_state=RANDOM_STATE, return_train_score=True)
    rand_search.fit(X_train, Y_train.values.ravel())
    
    # Armazenar histórico de desempenho
    mean_test_scores = rand_search.cv_results_['mean_test_score']
    std_test_scores = rand_search.cv_results_['std_test_score']
    performance_history.append((mean_test_scores, std_test_scores))
    
    best_mlps.append(('mlp' + str(i+1), rand_search.best_estimator_))

In [ ]:
# Criando o comitê com votação majoritária
voting_clf = VotingClassifier(estimators=best_mlps, voting='hard')

In [ ]:
# Treinando o comitê
voting_clf.fit(X_train, Y_train.values.ravel())

In [ ]:
# Fazendo previsões
Y_pred = voting_clf.predict(X_val)

In [ ]:
# Avaliação do modelo
accuracy = accuracy_score(Y_val, Y_pred)
print(f'Acurácia do Comitê de Redes Neurais: {accuracy:.4f}')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
# Plotar a evolução dos resultados de busca
plt.figure(figsize=(10, 5))
for i, (mean_scores, std_scores) in enumerate(performance_history):
    plt.plot(range(1, len(mean_scores) + 1), mean_scores, marker='o', label=f'MLP {i+1}')
    plt.fill_between(range(1, len(mean_scores) + 1), 
                     np.array(mean_scores) - np.array(std_scores), 
                     np.array(mean_scores) + np.array(std_scores), alpha=0.2)

plt.xlabel("Iteração")
plt.ylabel("Acurácia Média")
plt.title("Evolução dos Resultados da Busca de Hiperparâmetros")
plt.legend()
plt.show()

## Analises no conjunto de teste

In [ ]:
# Fazendo previsões no conjunto de teste
Y_pred_test = voting_clf.predict(X_test)
Y_prob_test = voting_clf.predict_proba(X_test)[:, 1]


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve

# Avaliação no conjunto de teste
accuracy = accuracy_score(Y_test, Y_pred_test)
precision = precision_score(Y_test, Y_pred_test)
recall = recall_score(Y_test, Y_pred_test)
f1 = f1_score(Y_test, Y_pred_test)
auc_roc = roc_auc_score(Y_test, Y_prob_test)

In [ ]:
print("Desempenho no conjunto de teste:")
print(f'Acurácia: {accuracy:.4f}')
print(f'Precisão: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')
print(f'AUC-ROC: {auc_roc:.4f}')

In [ ]:
# Matriz de confusão
import seaborn as sns

conf_matrix = confusion_matrix(Y_test, Y_pred_test)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Previsto")
plt.ylabel("Real")
plt.title("Matriz de Confusão")
plt.show()

In [ ]:
# Curva ROC
fpr, tpr, _ = roc_curve(Y_test, Y_prob_test)
plt.plot(fpr, tpr, label=f'AUC = {auc_roc:.4f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("Falso Positivo")
plt.ylabel("Verdadeiro Positivo")
plt.title("Curva ROC")
plt.legend()
plt.show()
